In [157]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
import cv2

# paths (specified by user)
imgs = PATH_TO_IMAGES
labs = PATH_TO_VOC_LABELS
direc_ids = PATH_TO_IMAGE_IDS_TXT_FILE
store_path = PATH_TO_STORE_EXTRACTED_LOGOS

with open(direc_ids) as f:
    # read in ids as list
    direc_ids = f.read().split('\n')

def xml_to_list(root, path, f = int):
    return list(map(lambda x: f(x.text), root.findall(path)))

def extract_logos(store_path, direc_ids):
    for i in direc_ids:
        img = imgs + i + ".png"
        im0 = cv2.imread(img) # read image
        lab = labs + i + ".xml"
        if os.path.exists(lab): # if label exists for image
            root = ET.parse(lab).getroot() 
            names = xml_to_list(root, './object/name', str) # annotation labels
            # bounding box coordinates
            x1s = xml_to_list(root, './object/bndbox/xmin')
            y1s = xml_to_list(root, './object/bndbox/ymin')
            x2s = xml_to_list(root, './object/bndbox/xmax')
            y2s = xml_to_list(root, './object/bndbox/ymax')
            for name, x1, y1, x2, y2 in zip(names, x1s, y1s, x2s, y2s):
                logo = im0[y1:y2, x1:x2] # extract logo using bbox coordinates
                store = store_path + name + "/"
                if os.path.exists(store) == False:
                    os.mkdir(store) # make store directory is doesn't exist
                num = str(len(os.listdir(store)) + 1)
                cv2.imwrite(store + num + ".png", logo) # write logo image
                
extract_logos(store_path, direc_ids)